# Site-based NMSIM models with Python

Davyd_Betchkal@nps.gov ◘ 2020-10-29

**TO DO**
>document all functions <br>
>more general timezone handling (preferrably from site coordinates!) <br>
>residual time glitches? (back to the *.trj* generating script on rds) <br>
>~comparison plot x-axis to datetimes~ <br>
>~does a weather file work?~ <font color="green">YES</font> ~if so, does it toggle to Nord2000 and change the quality of the results?~ <font color="red">NO</font> <br>
>add some model information to output of `NMSIM_create_tis` <br>
>*combine with other script?* <font color="green">YES</font>

In [1]:
import pandas as pd
pd.options.display.float_format = '{:.5f}'.format
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from itertools import islice
import datetime as dt
import re
import datetime
import glob
import os
import shutil
import subprocess
import sys

# # load iyore
# sys.path.append(r"C:\Users\ahug\Documents\PythonScripts\iyore")
import iyore

sys.path.append(r"C:\Users\DBetchkal\PythonScripts\3 GITHUB REPOSITORIES\soundDB")
# sys.path.append(r"C:\Users\ahug\Documents\PythonScripts\soundDB")
from soundDB import *

# import query tracks from RDS
sys.path.append(r"L:\scripts")
import query_tracks


# =================== DEFINE FUNCTIONS ==========================

def NMSIM_create_tis(project_dir, source_path, Nnumber=None, NMSIMpath=None):
    
    '''
    Create a site-based model run (.tis) using the NMSIM batch processor.
    
    Inputs
    ------
    
    Returns
    -------
    
    None
    
    '''
    
    # ======= (1) define obvious, one-to-one project files ================
    
    elev_file = project_dir + os.sep + r"Input_Data\01_ELEVATION" + os.sep + "elevation.flt"
    
    # imped_file = project_dir + os.sep + "Input_Data\01_IMPEDANCE" + os.sep + "landcover.flt"
    imped_file = None

    trj_files = glob.glob(project_dir + os.sep + r"Input_Data\03_TRAJECTORY\*.trj")

    # eventually the batch file is going to want this
    tis_out_dir = project_dir + os.sep + r"Output_Data\TIG_TIS"
    
    # ======= (2) define less obvious project files - these still need thought! ================
    
    # site files need some thinking through... there COULD be more than one per study area
    # (it's quite project dependant)
    site_file = glob.glob(project_dir + os.sep + r"Input_Data\05_SITES\*.sit")[0]
    
    # strip out the FAA registration number
    registrations = [t.split("_")[-3][11:] for t in trj_files]

    # the .tis name preserves: reciever + source + time (roughly 'source : path : reciever')
    site_prefix = os.path.basename(site_file)[:-4]

    tis_files = [tis_out_dir + os.sep + site_prefix + "_" + os.path.basename(t)[:-4] for t in trj_files]

    trajectories = pd.DataFrame([registrations, trj_files, tis_files], index=["N_Number","TRJ_Path","TIS_Path"]).T
    
    # ======= (3) write the control + batch files for command line control of NMSIM ================
        
    # set up the two files we want to write
    control_file = project_dir + os.sep + "control.nms"
    batch_file = project_dir + os.sep + "batch.txt"
    
    # select the trajectories to process
    if(Nnumber == None):

        trj_to_process = trajectories
    
    else:

        trj_to_process = trajectories.loc[trajectories["N_Number"] == Nnumber, :]
    
    
    if(NMSIMpath == None):
        
        # assume that the project folder is in "..\NMSIM_2014\Data"
        # and look for Nord2000batch.exe two directories up
        Nord = os.path.dirname(os.path.dirname(project_dir)) + os.sep + "Nord2000batch.exe"
        
    else:
        
        Nord = NMSIMpath

    for meta, flight in trj_to_process.iterrows():
    

        # write the control file for this situation
        with open(control_file, 'w') as nms:

            nms.write(elev_file+"\n") # elevation path
            
            if(imped_file != None):
                nms.write(imped_file+"\n") # impedance path
            else:
                nms.write("-\n")
                
            nms.write(site_file+"\n") # site path
            nms.write(flight["TRJ_Path"]+"\n")
            nms.write("-\n")
            nms.write("-\n")
            nms.write(source_path+"\n")
            nms.write("{0:11.4f}   \n".format(500.0000))
            nms.write("-\n")
            nms.write("-")    

        # write the batch file to create a site-based analysis
        with open(batch_file, 'w') as batch:

            batch.write("open\n")
            batch.write(control_file+"\n")
            batch.write("site\n")
            batch.write(flight["TIS_Path"]+"\n")
            batch.write("dbf: no\n")
            batch.write("hrs: 0\n")
            batch.write("min: 0\n")
            batch.write("sec: 0.0")
        
        # ======= (4) compute the theoretically observed trace on the site's microphone ================
        
        print(flight["TRJ_Path"]+"\n")
        
        process = subprocess.Popen([Nord, batch_file], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        stdout, stderr = process.communicate()

        output_messages = stdout.decode("utf-8").split("\r\n")
        output_messages = [ out for out in output_messages if out.strip() != '' ]
        
        print("\tthe following lines are directly from NMSIM:")
        for s in output_messages+["\n"]:
            print("\t"+s)

        # slightly messier printing for error messages
        if(stderr != None):
            for s in sterr.decode("utf-8").split("\r\n"):
                print(s.strip()) 
                

def pair_trj_to_tis_results(project_dir):
    
    '''
    Join a directory of .tis results created by NMSIM
      to the .trj files that created them.
      
    Inputs
    ------
    project_dir (str): the path to a canonical NPS-style NMSIM project directory
    
    Returns
    -------
    iterator (zip object): an iterator containing the paired .tis and .trj file paths
    
    '''
    
    # find all the '.tis' files
    successful_tis = glob.glob(project_dir + os.sep + "Output_Data\TIG_TIS\*.tis")

    # find all the '.trj' files
    trajectories = [project_dir + os.sep + "Input_Data\\03_TRAJECTORY" + \
                    os.sep + os.path.basename(f)[9:-4] + ".trj" for f in successful_tis]
    
    iterator = zip(trajectories, successful_tis)
    
    return iterator


def tis_resampler(tis_path, dt_start, utc_offset=-8):
    
    '''
    '''
    
    # read the data line-by-line
    with open(tis_path) as f:

        content = list(islice(f, 18 + (3600*24)))
    
    
    # find the line index where the header ends
    splitBegin = content.index('---End File Header---\n')

    # take out the whitespace and two empty columns at either end
    spectral_data = [re.split(r'\s+',c) for c in content[splitBegin+10:]] 
    spectral_data = [d[1:-2] for d in spectral_data]      
    
    # initalize a pandas dataframe using the raw spectral data and the expected column headers
    tis = pd.DataFrame(spectral_data, columns=["SP#","TIME","F","A","10", "12.5","15.8","20","25","31.5","40","50","63",
                                                "80","100","125","160","200","250","315","400","500","630","800","1000",
                                                "1250","1600","2000","2500","3150","4000","5000","6300","8000","10000",
                                                "12500"], dtype='float') #,"20000"

    # there's a weird text line at the end of the file (is this true for all .tis files?)
    tis.drop(tis.tail(1).index,inplace=True) # drop last n rows

    # these columns are stubborn
    tis["TIME"] = tis["TIME"].astype('float')
    tis["SP#"] = tis["SP#"].astype('float').apply(lambda f: int(f))
    tis["F"] = tis["F"].astype('int')

    # convert relevant columns to decibels (dB) from centibels (cB)
    tis.loc[:,'A':'12500'] *= 0.1

    # timedelta to adjust to local time
    utc_offset = dt.timedelta(hours=utc_offset) 

    # reindex the dataframe to AKT
    tis.index = tis["TIME"].astype('float').apply(lambda t: dt_start + dt.timedelta(seconds=t) + utc_offset)

    # resample to match NVSPL time resolution
    clean_tis = tis.sort_index().resample('1S').quantile(0.5)
    
    return clean_tis


def NVSPL_to_match_tis(ds, clean_tis, trj, unit, site, year, utc_offset=-8, pad_length=5):
    
    '''
    '''
    
    # timedelta to adjust to local time
    utc_offset = dt.timedelta(hours=utc_offset) 
    
    # convert startdate to Alaska Time
    ak_start = startdate + utc_offset
    
    # tidy up the TIS spectrogram by converting np.nan to -99.9
    clean_tis.fillna(-99.9).values.T
    
    # we can only compare 1/3rd octave bands down to 12.5 Hz... drop the rest
    clean_tis = clean_tis.loc[:, ~clean_tis.columns.isin(["SP#", "TIME", "F", "A", "10"])]

    # load NVSPL for the day of the event
    nv = nvspl(ds,
               unit=unit,
               site=site,
               year=ak_start.year,
               month=str(ak_start.month).zfill(2),
               day=str(ak_start.day).zfill(2),
               hour=[str(h).zfill(2) for h in np.unique(clean_tis.index.hour.values)],
               columns=["H"+s.replace(".", "p") for s in clean_tis.columns]).combine()

    # if multiple hours, drop the heirarchical index
    if isinstance(nv.index, pd.MultiIndex):
        nv.index = nv.index.droplevel(0)

    # select the SPL data that corresponds 
    pad = dt.timedelta(minutes=pad_length)
    
    # find the NVSPL data the specifically corresponds to the timing of the model
    event_SPL = nv.loc[clean_tis.index[0]-pad : clean_tis.index[-1]+pad,:]
    
    print("NVSPL shape:", event_SPL.shape)

    # pad the theoretical data as well
    spect_pad = np.full((int(pad.total_seconds()), clean_tis.shape[1]), -99.9)
    theoretical = np.vstack((spect_pad, clean_tis))

    print("NMSIM shape:", theoretical.shape)
    
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(18,5), sharex=True)
    
    # convert the NVSPL's nice datetime axis to numbers
    x_lims = mdates.date2num(event_SPL.index)
    
    ax[0].set_title("NMISIM results", loc="left")
    ax[0].imshow(theoretical.T, aspect='auto', origin='lower', 
                 extent=[x_lims[0], x_lims[-1], 0, event_SPL.shape[1]],
                 cmap='plasma', interpolation=None, vmin=-10, vmax=80, zorder=-5)

    ax[0].set_yticks(np.arange(event_SPL.shape[1])[::4])
    ax[0].set_yticklabels(event_SPL.columns.astype('float')[::4])
    
    # tell matplotlib that the numeric axis should be formatted as dates
    ax[0].xaxis_date()
    ax[0].xaxis.set_major_formatter(mdates.DateFormatter("%b-%d\n%H:%M")) # tidy them!
    
    ax[1].set_title("microphone measurement at "+unit+site, loc="left")
    im = ax[1].imshow(event_SPL.T, aspect='auto', origin='lower', 
                      extent=[x_lims[0], x_lims[-1], 0, event_SPL.shape[1]],
                      cmap='plasma', interpolation=None, vmin=-10, vmax=80)
    
    # the same as for the first plot
    ax[1].set_yticks(np.arange(event_SPL.shape[1])[::4])
    ax[1].set_yticklabels(event_SPL.columns.astype('float')[::4])
    ax[1].xaxis_date()
    ax[1].xaxis.set_major_formatter(mdates.DateFormatter("%b-%d\n%H:%M")) # tidy them!

    fig.colorbar(im, ax=ax.ravel().tolist(), anchor=(2.2, 0.0))
    fig.text(1.06, 0.5, "Sound Level (Leq, 1s)", va='center', rotation='vertical', fontsize=10)
    fig.text(-0.02, 0.55, "Frequency Band (Hz)", va='center', rotation='vertical', fontsize=13)
    
    title = os.path.basename(trj)[:-4]
    
    plt.suptitle(title, y=1.05, fontsize=17, ha="center")

    fig.tight_layout()

    plt.savefig(project_dir + os.sep + r"Output_Data\IMAGES" + os.sep + title + "_comparison.png", dpi=300, bbox_inches="tight")
    plt.show()
    
    return event_SPL


### Write a control file 'from scratch'

>*elevation_path* <br>
>*impedance_path* (or empty) <br>
>*site_path* <br>
>*trajectory_path* <br>
>*-* <br>
>*-* <br>
>*source_path* <br>
>*contour interval (m) as* `"{0:11.4f}   ".format(500.0000)` <br>
>*-* <br>
>*-* <br>

### Write a batch file 'from scratch'

>**open** <br>
>*control_file_path* <br>
>**site** <br>
>*tis_file_output* (no extension) <br>
>**dbf: no** <br>
>**hrs: 0** <br>
>**min: 0** <br>
>**sec: 0.0** <br>


## Run the models in NMSIM
#### Repeat for each N-Number, *then* proceed to the next cell.


In [3]:
# ============= EDIT THESE ===================================================

# project directory for the current site of interest
# project_dir = r"C:\Users\ahug\Documents\NMSim_2014\Data\DENAPRM4"
project_dir = r"C:\Users\DBetchkal\Desktop\NMSIM_2014_local\Data\DENASAN4"

# FAA registry for aircraft of interest
Focal_NNumber = "N21HY"

# ============================================================================

# extract the site name from the project directory path
site = project_dir[-4:]

# NMSIM program directiory
NMSIMpath = os.path.dirname(os.path.dirname(project_dir))

# lookup for appropriate NMSIM source file
source_map = {"N8888": NMSIMpath + os.sep + "Sources\MiscellaneousSources\omni.src",
              "N709M": NMSIMpath + os.sep + "Sources\AirTourFixedWingSources\C182.src",
              "N570AE": NMSIMpath + os.sep + "Sources\AirTourHelicopterSources\AS350.src",
               "N74PS": NMSIMpath + os.sep + "Sources\AirTourFixedWingSources\C207.src",
              "N619CH": NMSIMpath + os.sep +  "Sources\AirTourFixedWingSources\C207.src",
              "N72309": NMSIMpath + os.sep + "Sources\AirTourFixedWingSources\C207.src",
              "N72395": NMSIMpath + os.sep + "Sources\AirTourFixedWingSources\C207.src",
              "N473YC": NMSIMpath + os.sep + "Sources\AirTourFixedWingSources\C207.src",
              "N21HY":  NMSIMpath + os.sep + "Sources\AirTourFixedWingSources\C182.src"}


# run the NMSIM model
NMSIM_create_tis(project_dir, source_map[Focal_NNumber], Nnumber=Focal_NNumber)

C:\Users\DBetchkal\Desktop\NMSIM_2014_local\Data\DENASAN4\Input_Data\03_TRAJECTORY\N21HY_20190401_220322.trj

	the following lines are directly from NMSIM:
	C:\Users\DBetchkal\Desktop\NMSIM_2014_local\Nord2000batch.exe                  
	Opening control file C:\Users\DBetchkal\Desktop\NMSIM_2014_local\Data\DENASAN4\control.nms                                                                                                                                                                                          
	 Generating site files
	DENAHOG4
	

C:\Users\DBetchkal\Desktop\NMSIM_2014_local\Data\DENASAN4\Input_Data\03_TRAJECTORY\N21HY_20190410_172827.trj

	the following lines are directly from NMSIM:
	C:\Users\DBetchkal\Desktop\NMSIM_2014_local\Nord2000batch.exe                  
	Opening control file C:\Users\DBetchkal\Desktop\NMSIM_2014_local\Data\DENASAN4\control.nms                                                                                                                        

## Load NMSIM results and pair with NPS acoustic measurements

In [3]:
# iyore dataset for access to the measurements
archive = iyore.Dataset(r"E:") # define a dataset object)

# NMSIM isn't always successful - we want to iterate only through files that WERE created
trj_and_tis = pair_trj_to_tis_results(project_dir)

diagnostic = True

for trj, tis in trj_and_tis:
    
    runName = os.path.basename(trj)[:-4]
    
    # read just the header of the trajectory
    with open(trj) as lines:
        head = [next(lines) for x in range(12)]

    # here's the string containing the starting time (to the second!)
    dateString = head[-1][-24:-5]

    # convert to a datetime object
    startdate = dt.datetime.strptime(dateString, "%Y-%m-%d %H:%M:%S")

    print("now working on run:", runName)
    
    try:
        # this is the theoretical 1/3rd octave band trace
        theory = tis_resampler(tis, startdate)
        
    except:
        print("tis", tis)
        print("produced a ValueError related to nan values in the .tis file")
    
    
    # compare and save results
    event_SPL = NVSPL_to_match_tis(archive, theory, trj, 
                                   unit="DENA", site=site, year=int(startdate.year),
                                   utc_offset=-8, pad_length=5)
    
    # show diagnostic plots
    if(diagnostic == True):
        
        plt.figure(figsize=(14, 3))
        plt.plot(theory.index, theory["A"], zorder=5, ls="", marker="o", ms=2)
        plt.legend(loc="best")
        plt.show()
            

FileNotFoundError: [Errno 2] No such file or directory: 'E:.structure.txt'